In [1]:
!pip install openpyxl pandas


In [2]:
from google.colab import files
import pandas as pd
import io

uploaded = files.upload()
input_file = list(uploaded.keys())[0]  # grab uploaded file name


Saving input_data_tt.xlsx to input_data_tt.xlsx


In [3]:
xls = pd.ExcelFile(input_file)
df_tt   = xls.parse("in_timetable")
df_cr   = xls.parse("in_course_roll_mapping")
df_rn   = xls.parse("in_roll_name_mapping")
df_room = xls.parse("in_room_capacity")


In [4]:
buffer = 5  # seats to leave in each room
density = 'Sparse'  # 'Sparse' or 'Dense'


In [5]:
import math

df_cr.columns = df_cr.columns.str.strip()
df_cr['course_code'] = df_cr['course_code'].str.upper().str.strip()
df_cr['rollno']      = df_cr['rollno'].str.upper().str.strip()

df_rn.columns = df_rn.columns.str.strip()
df_rn['Roll'] = df_rn['Roll'].str.upper().str.strip()

name_dict = pd.Series(df_rn.Name.values, index=df_rn.Roll).to_dict()
course_to_rolls = df_cr.groupby('course_code')['rollno'].apply(lambda s: sorted(s.tolist())).to_dict()

rooms = []
for _,r in df_room.iterrows():
    rid = str(r['Room No.']).strip()
    cap = int(r['Exam Capacity'])
    block = str(r['Block']).strip()
    num = int(rid) if block=='B1' else int(rid.split('-')[-1])
    rooms.append(dict(room=rid, capacity=cap, block=block, numeric=num))


In [6]:
def allocate_course(course, rolls, avail):
    N = len(rolls)
    allowed=[]
    for r in avail:
        eff = r['capacity'] - buffer
        if eff <= 0: continue
        use = math.floor(eff*0.5) if density == 'Sparse' else eff
        if use > 0:
            allowed.append(dict(**r, allowed=use))
    if sum(r['allowed'] for r in allowed) < N:
        return None
    b1 = sorted([r for r in allowed if r['block']=='B1'], key=lambda x:x['numeric'])
    b2 = sorted([r for r in allowed if r['block']=='B2'], key=lambda x:x['numeric'])

    for pool in (b1, b2):
        if sum(r['allowed'] for r in pool) >= N:
            best = None
            for i in range(len(pool)):
                tot = 0
                for j in range(i, len(pool)):
                    tot += pool[j]['allowed']
                    if tot >= N:
                        L = j - i + 1
                        if not best or L < best[0]:
                            best = (L, i, j)
                        break
            i, j = best[1], best[2]
            seg = pool[i:j+1]
            alloc = []; rem = N; idx = 0
            for rr in seg:
                t = min(rem, rr['allowed'])
                alloc.append((rr['room'], rolls[idx:idx+t]))
                idx += t; rem -= t
            return alloc

    first, second = (b1, b2) if sum(r['allowed'] for r in b1) >= sum(r['allowed'] for r in b2) else (b2, b1)
    alloc = []; rem = N; idx = 0
    for rr in first:
        if rem <= 0: break
        t = min(rem, rr['allowed'])
        alloc.append((rr['room'], rolls[idx:idx+t]))
        idx += t; rem -= t
    for rr in second:
        if rem <= 0: break
        t = min(rem, rr['allowed'])
        alloc.append((rr['room'], rolls[idx:idx+t]))
        idx += t; rem -= t
    return None if rem > 0 else alloc


In [7]:
overall = []; seats_left = []
morning_by_date = {}; evening_by_date = {}

for _, r in df_tt.iterrows():
    date = pd.to_datetime(r['Date']).strftime("%Y-%m-%d")
    morn = [] if pd.isna(r['Morning']) else [c.strip() for c in r['Morning'].split(';')]
    eve  = [] if pd.isna(r['Evening']) else [c.strip() for c in r['Evening'].split(';')]

    avail = rooms.copy(); overflow = []
    for c in sorted(morn, key=lambda x: -len(course_to_rolls.get(x, []))):
        rolls = course_to_rolls.get(c, [])
        if not rolls: continue
        alloc = allocate_course(c, rolls, avail)
        if not alloc: overflow.append(c)
        else:
            for rm, grp in alloc:
                avail = [x for x in avail if x['room'] != rm]
                overall.append({'Date': date, 'Course': c, 'Room': rm, 'Count': len(grp), 'Rolls': ";".join(grp)})
                morning_by_date.setdefault(date, []).append((c, rm, grp))

    avail = rooms.copy()
    for c in sorted(eve+overflow, key=lambda x: -len(course_to_rolls.get(x, []))):
        rolls = course_to_rolls.get(c, [])
        if not rolls: continue
        alloc = allocate_course(c, rolls, avail)
        if not alloc:
            seats_left.append({'Date': date, 'Course': c, 'Unallocated': len(rolls)})
        else:
            for rm, grp in alloc:
                avail = [x for x in avail if x['room'] != rm]
                overall.append({'Date': date, 'Course': c, 'Room': rm, 'Count': len(grp), 'Rolls': ";".join(grp)})
                evening_by_date.setdefault(date, []).append((c, rm, grp))


In [8]:
df_overall = pd.DataFrame(overall)
df_left = pd.DataFrame(seats_left)

df_overall.to_excel("op_overall.xlsx", index=False)
df_left.to_excel("op_seats_left.xlsx", index=False)


In [9]:
from openpyxl import Workbook, load_workbook

def build_workbook(routines, session):
    wb = Workbook()
    ws = wb.active
    ws.title = "Dummy"
    for date, blocks in routines.items():
        ws = wb.create_sheet(title=date)
        row = 1
        for course, room, grp in blocks:
            title = f"Course: {course} | Room: {room} | Date: {date} | Session: {session}"
            ws.cell(row=row, column=1, value=title)
            row += 1
            ws.append(["Roll", "Student Name", "Signature"])
            row += 1
            for rn in grp:
                nm = name_dict.get(rn, "Unknown Name")
                ws.append([rn, nm, ""])
                row += 1
            row += 2
    wb.remove(wb["Dummy"])
    return wb

wb_morning = build_workbook(morning_by_date, "Morning")
wb_evening = build_workbook(evening_by_date, "Evening")

wb_morning.save("morning_routines.xlsx")
wb_evening.save("evening_routines.xlsx")


In [10]:
from google.colab import files
files.download("op_overall.xlsx")
files.download("op_seats_left.xlsx")
files.download("morning_routines.xlsx")
files.download("evening_routines.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
from google.colab import files
files.download("op_overall.xlsx")
files.download("op_seats_left.xlsx")
files.download("morning_routines.xlsx")
files.download("evening_routines.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>